# Предобработка телеметрии самосвалов (продолжение)
Добавление дополнительной информации из разных таблиц





In [1]:
import os
from pathlib import Path

import tqdm
import warnings

import pandas as pd
from pandas.api.types import is_numeric_dtype

### Функция оптимизации памяти

In [2]:
def optimize_dtypes(df):
    df_optimized = df.copy()

    # float64 → float32
    float_cols = df_optimized.select_dtypes(include='float64').columns
    df_optimized[float_cols] = df_optimized[float_cols].astype('float32')

    # int64 → int32
    int_cols = df_optimized.select_dtypes(include='int64').columns
    df_optimized[int_cols] = df_optimized[int_cols].astype('int32')

    # object → category (для строк, где мало уникальных значений)
    object_cols = df_optimized.select_dtypes(include='object').columns
    for col in object_cols:
        num_unique_values = df_optimized[col].nunique()
        num_total_values = len(df_optimized[col])
        if num_unique_values / num_total_values < 0.5:
            df_optimized[col] = df_optimized[col].astype('category')

    return df_optimized


def setup_pandas_options():
    """Настройка глобальных опций pandas для отображения."""
    pd.set_option("display.precision", 3)
    pd.set_option("expand_frame_repr", False)


setup_pandas_options()

In [3]:
data_root = '../dataset'
source_root = '../dataset/_by_Hack'

## 2. Загрузка и объединение данных

In [4]:
telemetry = pd.read_parquet(f'{data_root}/raw/telemetry/telemetry_filtered_optimized.parquet')
equipment = pd.read_csv(f'{source_root}/reference/equipment.csv')
idles = pd.read_csv(f'{source_root}/reference/idles.csv')
oil = pd.read_csv(f'{source_root}/oil/oil.csv')
print('telemetry:', telemetry.shape)
print('equipment:', equipment.shape)
print('idles:', idles.shape)
print('oil_lab:', oil.shape)

telemetry: (34314060, 24)
equipment: (12, 7)
idles: (585830, 35)
oil_lab: (147, 39)


In [5]:
# 4. Преобразование дат
# Телеметрия
telemetry['create_dt'] = pd.to_datetime(telemetry['create_dt'])
# Масло — используем TakenDate, если есть, иначе ReportedDate
time_col = 'TakenDate' if 'TakenDate' in oil.columns else 'ReportedDate'
oil['sample_time'] = pd.to_datetime(oil[time_col].fillna(oil.get('ReportedDate')))
# Простои
idles['GMTBEGINTIME'] = pd.to_datetime(idles['GMTBEGINTIME'])
idles['GMTENDTIME'] = pd.to_datetime(idles['GMTENDTIME'])

In [6]:
# Optimize memory
telemetry = optimize_dtypes(telemetry)
equipment = optimize_dtypes(equipment)
idles = optimize_dtypes(idles)
oil = optimize_dtypes(oil)

In [7]:
# 5. Создание общего ключа truck_number

# ---------- МАСЛО: truck_number ----------
if 'UnitNumberField' in oil.columns:
    unit_raw = oil['UnitNumberField']

    if is_numeric_dtype(unit_raw):
        # Всё хорошо, числа: 100.0, 1374.0 и т.п.
        oil['truck_number'] = unit_raw.astype('Int64').astype(str)
    else:
        # Смешанные / строковые значения: пробуем вытащить числа
        # 1) попытка привести к числам
        unit_num = pd.to_numeric(unit_raw, errors='coerce')

        # 2) где получилось — используем как номер борта
        truck_number_from_unit = unit_num.astype('Int64').astype(str)

        # 3) fallback: берём номер из CustUnitIdField (последнее слово)
        if 'CustUnitIdField' in oil.columns:
            fallback_from_cust = (
                oil['CustUnitIdField']
                .astype(str)
                .str.split()
                .str[-1]          # "BELAZ 75306 1374" -> "1374"
            )
        else:
            fallback_from_cust = pd.Series(index=oil.index, dtype='object')

        # 4) если unit_num не получилось (NaN/<NA>), подставляем fallback
        oil['truck_number'] = truck_number_from_unit.mask(
            truck_number_from_unit.isin(['<NA>', 'nan', 'NaN']),
            fallback_from_cust
        )

else:
    # Вообще нет UnitNumberField — сразу из CustUnitIdField
    oil['truck_number'] = (
        oil['CustUnitIdField']
        .astype(str)
        .str.split()
        .str[-1]
    )

# На всякий случай: truck_number как строка
oil['truck_number'] = oil['truck_number'].astype(str)

# ---------- ТЕЛЕМЕТРИЯ: truck_number ----------
telemetry['truck_number'] = telemetry['mdm_object_name'].astype(str)

# ---------- ПРОСТОИ: truck_number ----------
idles['truck_number'] = idles['OBJECTNAME'].astype(str)

In [ ]:
# Объединение телеметрии с маслами
merged_data = telemetry.merge(oil[['truck_number', 'ReportedDate', 'Condition', 'OilType']], on='truck_number', how='left')
# Объединение с данными о простоях
final_data = merged_data.merge(idles[['truck_number', 'GMTBEGINTIME', 'GMTENDTIME']], on='truck_number', how='left')
# Проверим результат
print(final_data.head())

In [ ]:
print(final_data)

In [46]:
# Объединяем по mdm_object_id
telemetry_data = telemetry.merge(equipment[['mdm_object_id', 'mdm_object_name', 'meta_model_name']], on='mdm_object_id', how='left')
full_data = telemetry_data.merge(oil[['UnitRef', 'ReportedDate', 'Condition', 'OilType']], left_on='mdm_object_id', right_on='UnitRef', how='left')
print('full_data:', full_data.shape)

full_data: (68628120, 30)


In [47]:
print(full_data)

                         create_dt mdm_object_id  mdm_object_name_x       mdm_model_name    alt  speed_gps  inst_fuel  weight  temp_engine  turn_engine  ...                       mdm_object_uuid  meta_object_name    meta_model_name_x  is_stopped  mdm_object_name_y         meta_model_name_y  UnitRef  ReportedDate  Condition  OilType
0        2024-01-01 12:23:42+11:00          1661               1395  БелАЗ 75306 Cummins  -66.0        0.0      293.0    19.0         66.0       6425.0  ...  83397e13-90c4-11ec-98b9-00155d5fc801              1395  БелАЗ 75306 Cummins           1               1395       БелАЗ 75306 Cummins      NaN           NaN        NaN      NaN
1        2024-01-01 12:23:42+11:00          1661               1395  БелАЗ 75306 Cummins  -66.0        0.0      293.0    19.0         66.0       6425.0  ...  83397e13-90c4-11ec-98b9-00155d5fc801              1395  БелАЗ 75306 Cummins           1               1395  БелАЗ 75306 Б GalileoSky      NaN           NaN        NaN      Na

Совпадений после преобразования: 0 из 68628120


### Как сделать надёжный ключ truck_number:
1. Если UnitNumberField числовой — используем его.
2. Если там строки — пытаемся вытащить из них число.
3. Если с числом не получилось — берём последнюю часть из CustUnitIdField.
4. Храним truck_number всегда как строку (и в масле, и в телеметрии, и в простоях).
---
#### Коротко логика
- Если UnitNumberField — нормальные числа: ОК.
- Если там строки ('A954', "100", "1374", NaN и пр.):
  - to_numeric(..., errors='coerce') превратит всё нечисловое в NaN;
  - для NaN мы берём номер из CustUnitIdField (в твоих примерах он красивый: BELAZ 75306 1374 → "1374").
- Везде приводим к str, чтобы не ловить неожиданности при merge.

In [13]:
# 5. Создание общего ключа truck_number

# ---------- МАСЛО: truck_number ----------
if 'UnitNumberField' in oil.columns:
    unit_raw = oil['UnitNumberField']

    if is_numeric_dtype(unit_raw):
        # Всё хорошо, числа: 100.0, 1374.0 и т.п.
        oil['truck_number'] = unit_raw.astype('Int64').astype(str)
    else:
        # Смешанные / строковые значения: пробуем вытащить числа
        # 1) попытка привести к числам
        unit_num = pd.to_numeric(unit_raw, errors='coerce')

        # 2) где получилось — используем как номер борта
        truck_number_from_unit = unit_num.astype('Int64').astype(str)

        # 3) fallback: берём номер из CustUnitIdField (последнее слово)
        if 'CustUnitIdField' in oil.columns:
            fallback_from_cust = (
                oil['CustUnitIdField']
                .astype(str)
                .str.split()
                .str[-1]          # "BELAZ 75306 1374" -> "1374"
            )
        else:
            fallback_from_cust = pd.Series(index=oil.index, dtype='object')

        # 4) если unit_num не получилось (NaN/<NA>), подставляем fallback
        oil['truck_number'] = truck_number_from_unit.mask(
            truck_number_from_unit.isin(['<NA>', 'nan', 'NaN']),
            fallback_from_cust
        )

else:
    # Вообще нет UnitNumberField — сразу из CustUnitIdField
    oil['truck_number'] = (
        oil['CustUnitIdField']
        .astype(str)
        .str.split()
        .str[-1]
    )

# На всякий случай: truck_number как строка
oil['truck_number'] = oil['truck_number'].astype(str)

# ---------- ТЕЛЕМЕТРИЯ: truck_number ----------
telemetry['truck_number'] = telemetry['mdm_object_name'].astype(str)

# ---------- ПРОСТОИ: truck_number ----------
idles['truck_number'] = idles['OBJECTNAME'].astype(str)


In [15]:
# 6. Оптимизация типов (после дат и ключей!)
telemetry = optimize_dtypes(telemetry)
idles = optimize_dtypes(idles)
oil_all = optimize_dtypes(oil_lab)

print(telemetry.info(memory_usage='deep'))
print(idles.info(memory_usage='deep'))
print(oil_lab.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34314060 entries, 0 to 34314059
Data columns (total 25 columns):
 #   Column                     Dtype                                
---  ------                     -----                                
 0   create_dt                  datetime64[ns, pytz.FixedOffset(660)]
 1   mdm_object_id              int32                                
 2   mdm_object_name            int32                                
 3   mdm_model_name             category                             
 4   alt                        float32                              
 5   speed_gps                  float32                              
 6   inst_fuel                  float32                              
 7   weight                     float32                              
 8   temp_engine                float32                              
 9   turn_engine                float32                              
 10  load_engine                float32      

In [16]:
# === 1. Приводим время к одному виду: datetime64[ns] без tz ===

# Телеметрия
telemetry['create_dt'] = pd.to_datetime(telemetry['create_dt'], utc=True)
telemetry['create_dt'] = telemetry['create_dt'].dt.tz_convert(None)

# Масло
time_col = 'TakenDate' if 'TakenDate' in oil_all.columns else 'ReportedDate'
oil_all['sample_time'] = pd.to_datetime(
    oil_all[time_col].fillna(oil_all.get('ReportedDate')),
    utc=True
)
oil_all['sample_time'] = oil_all['sample_time'].dt.tz_convert(None)

# === 2. Ключи по машине — точно строки ===

telemetry['truck_number'] = telemetry['truck_number'].astype(str)
oil_all['truck_number']   = oil_all['truck_number'].astype(str)

# === 3. Убираем строки без времени (NaT) ===

telemetry_no_na = telemetry.dropna(subset=['create_dt'])
oil_no_na       = oil_all.dropna(subset=['sample_time'])

# === 4. Подмножество столбцов телеметрии ===

telemetry_cols_for_merge = [
    'truck_number',
    'create_dt',
    'mdm_object_id',
    'meta_model_name',
    'temp_engine',
    'turn_engine',
    'load_engine',
    'inst_fuel',
    'speed_gps',
]
telemetry_cols_for_merge = [c for c in telemetry_cols_for_merge
                            if c in telemetry_no_na.columns]

# === 5. merge_asof ПО КАЖДОМУ БОРТУ ОТДЕЛЬНО ===

merged_chunks = []

# группируем масло по борту
for truck, oil_grp in oil_no_na.groupby('truck_number'):
    # телеметрия для этого борта
    tel_grp = telemetry_no_na[telemetry_no_na['truck_number'] == truck]

    if tel_grp.empty:
        # если для этого борта нет телеметрии — просто добавляем масло как есть
        merged_chunks.append(oil_grp.copy())
        continue

    # сортировка по времени (внутри одного борта теперь точно будет ОК)
    oil_grp_sorted = oil_grp.sort_values('sample_time')
    tel_grp_sorted = tel_grp.sort_values('create_dt')[telemetry_cols_for_merge]

    merged = pd.merge_asof(
        oil_grp_sorted,
        tel_grp_sorted,
        left_on='sample_time',
        right_on='create_dt',
        direction='backward',
        allow_exact_matches=True,
    )

    merged_chunks.append(merged)

# Финальный датасет: масло + телеметрия
oil_telemetry = pd.concat(merged_chunks, ignore_index=True)

print('oil_telemetry (масло + телеметрия):', oil_telemetry.shape)


oil_telemetry (масло + телеметрия): (147, 50)


Предиктивная модель строится вокруг события контроля.
Поэтому правильно мержить именно масло → телеметрия, а не наоборот. Тк телеметрия — это time series с миллионами точек, а масло — это события раз в N часов.

Телеметрия:
- 34 314 060 строк
- поток сигналов раз в секунду/5 секунд

Масло:
- 147 строк
- раз в 250–500 моточасов

Если попытаться записать «масло в телеметрию», то получится:
 - либо 34 млн строк, где почти все значения масла NaN
 - либо неработоспособный датасет, где одно событие растянуто на сотни тысяч записей

Это полностью разрушит ML-постановку задачи.

Целевая переменная (Y) — это состояние узла, которое известно только в моменты анализа масла

Например:
- Condition = Normal / Abnormal / Severe
- или EvaluationComment = Acceptable / Unacceptable / Critical

То есть Y существует только в моменты масла.

А телеметрия должна быть агрегирована относительно момента пробы масла, например:
- средняя температура двигателя за последние 24 часа
- максимальная температура за последние 8 часов
- средняя нагрузка за последние 10 часов
- количество событий перегрева за последние 7 дней

Именно поэтому каждая строка в итоговом датасете должна соответствовать одной пробе масла.
И телеметрию тогда можно будет использовать как признаки (X).

In [17]:
display(oil_telemetry.head())

,SampleId,ComponentRef,UnitRef,CustomerId,ReportedDate,TakenDate,ReceivedDate,ViscMode,Particle,Parker,...,truck_number_x,truck_number_y,create_dt,mdm_object_id,meta_model_name,temp_engine,turn_engine,load_engine,inst_fuel,speed_gps
0,46632,ML200011QSK77,ML200011BEL745,ML200011OOO953,2023-04-14 11:51:42.000,2023-04-13 00:00:00.000,2023-04-13 00:00:00.000,Measured,False,True,...,1349,1349,2023-04-12 23:59:57,1381,БелАЗ 75306 Б GalileoSky,78.0,265.0,0.0,164.0,0.0
1,48879,ML200011GS 366,ML200011BEL745,ML200011OOO953,2023-05-01 16:39:53.000,2023-05-01 16:38:16.000,2023-05-01 16:38:16.000,Measured,True,True,...,1349,1349,2023-04-17 05:28:16,1381,БелАЗ 75306 Б GalileoSky,80.0,329.0,0.0,192.0,0.0
2,48880,ML200011QSK77,ML200011BEL745,ML200011OOO953,2023-05-01 17:02:54.000,2023-05-01 17:00:58.000,2023-05-01 17:00:58.000,Measured,False,True,...,1349,1349,2023-04-17 05:28:16,1381,БелАЗ 75306 Б GalileoSky,80.0,329.0,0.0,192.0,0.0
3,57708,ML200011QSK77,ML200011BEL745,ML200011OOO953,2023-07-01 14:51:37.000,2023-07-01 14:50:08.000,2023-07-01 14:50:08.000,Measured,False,True,...,1349,1349,2023-07-01 14:50:05,1381,БелАЗ 75306 Cummins,90.0,9002.0,0.0,77.0,36.6
4,64116,ML200011GS 366,ML200011BEL745,ML200011OOO953,2023-07-23 08:45:33.000,2023-07-20 00:00:00.000,2023-07-20 00:00:00.000,Measured,True,True,...,1349,1349,2023-07-20 00:00:00,1381,БелАЗ 75306 Cummins,78.0,13961.0,100.0,7784.0,16.6


In [18]:
# 9. (Опционально) Базовый справочник техники из телеметрии + простоев
equip_from_tel = telemetry[['mdm_object_id', 'mdm_object_name', 'meta_model_name', 'truck_number']].drop_duplicates()
equip_from_idle = idles[['OBJECTID', 'OBJECTNAME', 'MODELNAME', 'truck_number']].drop_duplicates()

equipment = equip_from_tel.merge(
    equip_from_idle,
    left_on='truck_number',
    right_on='truck_number',
    how='outer',
    suffixes=('_tel', '_idle')
)

print('equipment (справочник техники):', equipment.shape)
equipment.to_csv(os.path.join(source_root, 'equipment.csv'), index=False)
display(equipment)

equipment (справочник техники): (12, 7)


,mdm_object_id,mdm_object_name,meta_model_name,truck_number,OBJECTID,OBJECTNAME,MODELNAME
0,1381,1349,БелАЗ 75306 Cummins,1349,1381,1349,БелАЗ 75306 Б
1,1381,1349,БелАЗ 75306 Б GalileoSky,1349,1381,1349,БелАЗ 75306 Б
2,1383,1374,БелАЗ 75306 Cummins,1374,1383,1374,БелАЗ 75306 Б
3,1383,1374,БелАЗ 75306 Б GalileoSky,1374,1383,1374,БелАЗ 75306 Б
4,1581,1381,БелАЗ 75306 Cummins,1381,1581,1381,БелАЗ 75306 Б
5,1581,1381,БелАЗ 75306 Б GalileoSky,1381,1581,1381,БелАЗ 75306 Б
6,1384,1385,БелАЗ 75306 Cummins,1385,1384,1385,БелАЗ 75306 Б
7,1384,1385,БелАЗ 75306 Б GalileoSky,1385,1384,1385,БелАЗ 75306 Б
8,1661,1395,БелАЗ 75306 Cummins,1395,1661,1395,БелАЗ 75306 Б
9,1661,1395,БелАЗ 75306 Б GalileoSky,1395,1661,1395,БелАЗ 75306 Б


## Предобработка для ML

In [19]:
# 1. Берём рабочую копию
df = oil_telemetry.copy()

# 2. Строим target
df['target'] = (
    df['Condition']
    .astype(str)
    .str.lower()
    .isin(['abnormal', 'severe'])
).astype('int8')

# 3. Удаляем ненужные колонки
cols_to_drop = [
    'Condition',
    'EvaluationComment',
    'DiagnosticStatement',
    'ComponentRef',
    'CustUnitIdField',
    'UnitRef',
    'create_dt',
    'ReceivedDate',
    'ReportedDate',
    'TakenDate',
    'sample_time',
    'truck_number_x',
    'truck_number_y',
]

# выкинуть Unnamed-столбцы
junk_cols = [c for c in df.columns if 'Unnamed' in c]

# оставляем только те, которые реально есть
cols_to_drop = [c for c in cols_to_drop if c in df.columns]

df = df.drop(columns=cols_to_drop + junk_cols)

# 4. Категориальные фичи → category (УЖЕ ПО df)
cat_cols = df.select_dtypes(include='object').columns
for c in cat_cols:
    df[c] = df[c].astype('category')

# 5. Числовые фичи → float32
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].astype('float32')

# 6. Сохраняем ML-готовый датасет
out_path = Path('../dataset/ml_datasets/oil_ml_ready.parquet')
out_path.parent.mkdir(parents=True, exist_ok=True)
df.to_parquet(out_path, index=False)

print("Saved:", out_path)
print(df.shape)
print("Категориальных признаков:", len(cat_cols), "Числовых:", len(num_cols))
display(df.head())



Saved: ../dataset/ml_datasets/oil_ml_ready.parquet
(147, 38)
Категориальных признаков: 1 Числовых: 0


,SampleId,CustomerId,ViscMode,Particle,Parker,OilCapacityUnits,AnalysisMode,OilTime,OilWeight,OilBrand,...,ComponentMakeField,ComponentIdField,mdm_object_id,meta_model_name,temp_engine,turn_engine,load_engine,inst_fuel,speed_gps,target
0,46632,ML200011OOO953,Measured,False,True,L,Off-Road,400,15W40,G-PROFI,...,BELAZ,QSK60 1349,1381,БелАЗ 75306 Б GalileoSky,78.0,265.0,0.0,164.0,0.0,1
1,48879,ML200011OOO953,Measured,True,True,L,Off-Road,4000,ISO32,LUKOIL LT32,...,BELAZ,GS 75306 1349,1381,БелАЗ 75306 Б GalileoSky,80.0,329.0,0.0,192.0,0.0,1
2,48880,ML200011OOO953,Measured,False,True,L,Off-Road,400,15W40,G-PROFI,...,BELAZ,QSK60 1349,1381,БелАЗ 75306 Б GalileoSky,80.0,329.0,0.0,192.0,0.0,1
3,57708,ML200011OOO953,Measured,False,True,L,Off-Road,400,15W40,G-PROFI,...,BELAZ,QSK60 1349,1381,БелАЗ 75306 Cummins,90.0,9002.0,0.0,77.0,36.6,1
4,64116,ML200011OOO953,Measured,True,True,L,Off-Road,400,ISO32,LUKOIL LT32,...,BELAZ,GS 75306 1349,1381,БелАЗ 75306 Cummins,78.0,13961.0,100.0,7784.0,16.6,1
